In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
def f (x):
  return 3*x**2-4*x+5

In [3]:
f(3)

20

In [4]:
xs = np.arange(-5,5,0.25)
ys = f(xs)
plt.plot(xs,ys)

In [5]:
h = 0.0001
x = 1
print(f(x))
print(f(x+h))
(f(x+h)-f(x))/h

4
4.00020003


2.0003000000023974

In [6]:
a = 2.0
b = -3.0
c = 10.0
d = a*b+c
print(d)

4.0


In [7]:
h = 0.001
# inputs
a = 2.0
b = -3.0
c = 10.0
d1 = a*b+c
a+=h
d2 = a*b+c
print((d2-d1)/h)

-3.0000000000001137


In [8]:
class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out

  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')

    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward

    return out

  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')

    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward

    return out


  def backward(self):

    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [9]:
a = Value(2.0)
b = Value(3.0)
a-b

Value(data=-1.0)

In [10]:
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d * f; L.label = 'L'
L


Value(data=-8.0)

In [11]:
d._prev

{Value(data=-6.0), Value(data=10.0)}

In [13]:
pip install 
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right

  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

ModuleNotFoundError: No module named 'graphviz'

In [ ]:
draw_dot(L)

In [ ]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'
o.backward()
draw_dot(o)

In [ ]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
# ----
e = (2*n).exp()
e.label = 'e'
o = (e - 1) / (e + 1)
# ----
o.label = 'o'
o.backward()
draw_dot(o)

In [ ]:
o.backward()
draw_dot(o)

In [ ]:
L.grad = 1.0

In [ ]:
d.grad = -2.0
f.grad = 1.0


In [ ]:
# this is intersting L = d*f L = (e+c)*f so dL/de = f
# dL/de = dL/dd * dd/de and dd/de = 1 dL/dd *dd/dc
e.grad = -2.0
c.grad = -2.0
a.grad = e.grad *b.data
b.grad = e.grad *a.data

In [ ]:
a = Value(-2.0,label = 'a')
b = Value(-3.0,label = 'b')
d = a*b ; d.label = 'd'
e = a+b; e.label = 'e'
f = d*e ; f.label = 'f'

f.backward()
draw_dot(f)

### Usinf pytorch to verify our solution

In [ ]:
import torch

In [ ]:
x1 = torch.Tensor([2.0]).double() ; x1.requires_grad= True
x2 = torch.Tensor([0.0]).double() ; x2.requires_grad= True
w1 = torch.Tensor([-3.0]).double() ; w1.requires_grad= True
w2 = torch.Tensor([1.0]).double() ; w2.requires_grad= True
b  = torch.Tensor([6.88137]).double();b.requires_grad = True

n = w1*x1 + w2*x2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print('---')
print('x2', x2.grad.item())
print('x1', x1.grad.item())
print('w1',w1.grad.item())
print('w2',w2.grad.item())


In [ ]:
import numpy as np
import random

In [ ]:
class Neuron:

  def __init__(self,nin): # nin here is the number of inputs given to that neuron
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))

  def __call__(self,x):
    # here x are the inputs given to the neuron

    activation = sum((wi*x1 for wi,xi in zip(self.w,x)),self.b)
    out = activation.tanh()
    return out

class Layer:

  def __init__(self,nin,nn_in_layer):
    # nn_in_layer are the number of neurons i want in the layer nin is the number of inputs received by each neuron in that layer
    self.neurons = [Neuron(nin) for _ in range(nn_in_layer)]
  def __call__(self,x):
    outputs = [n(x) for n in self.neurons]
    return outputs if len(outputs) > 1 else outputs[0]

class MLP:

  def __init__(self,nin,nout):
    # nin is the number of inputs to out mulitlayer perceptron nout is a list where each element tells the number of neuron in that layer
    sz = [nin]+nout
    self.layers = [Layer(sz[i],sz[i+1])for i in range(len(nout))]

  def __call__(self,x):
      for l in self.layers:
        x = l(x)
      return x


In [ ]:
x = [2.0,3.0,-1.0]
nn = MLP(3,[4,4,1])
nn(x)

In [ ]:
l1 = Layer(2,3)
x = [1.0,2.3]
l1(x)

In [ ]:
draw_dot(nn(x))